<a href="https://colab.research.google.com/github/Fabriciooml/Hash-Functions/blob/main/hash_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown ##Defina o tamanho da tabela hash (recomendável ser um [número primo](https://en.wikipedia.org/wiki/List_of_prime_numbers))
size = 7 #@param {type:"integer"}
#@markdown ##Defina o incremento fixo
fixed_increment = 5 #@param {type:"integer"}

In [5]:
#@markdown ##Classe da tabela hash
from typing import Dict, List, Union

class hash_table:

  def __init__(self, size: int, fixed_increment: int) -> None:
    self.size = size
    self.length = 0
    self.increment = fixed_increment
    self.table: List[Union[None, Dict[int, int]]] = [None] * size

  def __str__(self) -> str:
    return f"{self.table}"

  def hash(self, key: int) -> int:
    return key % self.size

  def is_full(self) -> bool:
    if self.length == self.size:
      return True
    return False

  def is_occupied(self, index: int) -> bool:
    if self.table[index] is not None:
      return True
    return False

  def is_marked_available(self, index: int) -> bool:
    if self.is_occupied(index):
      if list(self.table[index].keys())[0] == "A":
        return True
    return False

  def insert(self, key: int, value: int) -> None:
    index = self.hash(key)
    if not self.is_full():
      while self.is_occupied(index) and not self.is_marked_available(index):
        print(f"Collision on index {index} using Linear Probing fixed increment unidirecional")
        index = self.hash(index + self.increment)

      self.table[index] = {key: value}
      print(f"inserted key {key} on index {index}")
      self.length += 1
    else:
      raise IndexError("Table is full")

  def find_index(self, key: int) -> int:
    index = self.hash(key)
    while self.is_occupied(index):
      if key in self.table[index].keys():
        return index
      index = self.hash(index + self.increment)
    raise KeyError("Key not found")

  def search(self, key:int) -> int:
    index = self.find_index(key)
    return self.table[index][key]

  def delete_from_index(self, old_index:int) -> None:
    new_index = self.hash(old_index + self.increment)

    while self.is_occupied(new_index):
      new_key = list(self.table[new_index].keys())[0]
      new_value = self.table[new_index][new_key]

      if new_key != "A":
        if self.hash(new_key) != new_index:
          self.table[old_index] = None
          self.insert(new_key, new_value)
          self.table[new_index] = None
          old_index = new_index
          self.length -= 1

      new_index = self.hash(new_index + self.increment)

    self.table[old_index] = None

  def set_as_available(self, index: int) -> None:
    if self.is_occupied(index):
      self.table[index] = {"A": "position available"}
      self.length -= 1

  def delete(self, key:int, use_available:bool = False) -> None:
    old_index = self.find_index(key)
    if not use_available:
      self.delete_from_index(old_index)
    else:
      self.set_as_available(old_index)

  def delete_available(self):
    for index in range(self.size):
      if self.is_occupied(index):
        if list(self.table[index].keys())[0] == "A":
          self.delete_from_index(index)

In [7]:
#@markdown ##Adicionando elementos na tabela hash
keys = "0, 1, 2, 3, 7, 14" #@param{type: "string"}
values = "4, 5, 6, 7, 10, 100" #@param{type: "string"}
keys_list = [int(x) for x in keys.strip().split(",")]
values_list = [int(x) for x in values.strip().split(",")]

table = hash_table(size, fixed_increment)

for i in range(len(keys_list)):
  table.insert(keys_list[i], values_list[i])

print(table)

inserted key 0 on index 0
inserted key 1 on index 1
inserted key 2 on index 2
inserted key 3 on index 3
Collision on index 0 using Linear Probing fixed increment unidirecional
inserted key 7 on index 5
Collision on index 0 using Linear Probing fixed increment unidirecional
Collision on index 5 using Linear Probing fixed increment unidirecional
Collision on index 3 using Linear Probing fixed increment unidirecional
Collision on index 1 using Linear Probing fixed increment unidirecional
inserted key 14 on index 6
[{0: 4}, {1: 5}, {2: 6}, {3: 7}, None, {7: 10}, {14: 100}]


In [8]:
#@markdown ## Fazendo busca

key = 0 #@param{type:"integer"}
print(f"O valor da key {key} é {table.search(key)}")

O valor da key 0 é 4


In [9]:
#@markdown ## Deletando elemento

key = 0 #@param{type:"integer"}
table.delete(key)
print(table)

inserted key 7 on index 0
Collision on index 0 using Linear Probing fixed increment unidirecional
inserted key 14 on index 5
[{7: 10}, {1: 5}, {2: 6}, {3: 7}, None, {14: 100}, None]


In [12]:
#@markdown ## Deletando elemento usando tag A

key = 7 #@param{type:"integer"}
table.delete(key, use_available=True)
print(table)

[{'A': 'position available'}, {'A': 'position available'}, {2: 6}, {3: 7}, None, {14: 100}, None]


In [13]:
#@markdown ## Inserindo elemento em local com tag A

key = 7 #@param{type:"integer"}
value = 99 #@param{type:"integer"}
table.insert(key, value)
print(table)

inserted key 7 on index 0
[{7: 99}, {'A': 'position available'}, {2: 6}, {3: 7}, None, {14: 100}, None]


In [14]:
#@markdown ## Apaga todos os elementos marcados com tag A

table.delete_available()
print(table)

[{7: 99}, None, {2: 6}, {3: 7}, None, {14: 100}, None]
